# Logistic Regression

A lot of this is going to be data cleaning in the beginning. 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
import statsmodels.api as sm



In [2]:
#read in our data
df = pd.read_csv('3yearsoriginal.csv', index_col='id')

C:\Users\hunte\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (19,59,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\hunte\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


I have several columns that I'm thinking of dropping for my project. Lets go ahead and get rid of them since I already have so many features. 

In [3]:
gendrops = ['member_id', 'funded_amnt', 'funded_amnt_inv', 'url','desc', 'title', 'policy_code', 'hardship_type', 
         'orig_projected_additional_accrued_interest','hardship_dpd', 'deferral_term', 'hardship_amount', 
         'hardship_start_date', 'hardship_end_date','payment_plan_start_date', 'hardship_length', 'hardship_loan_status', 
         'hardship_reason', 'hardship_payoff_balance_amount', 'hardship_payoff_balance_amount', 
         'hardship_last_payment_amount', 'hardship_type', 'hardship_status', 'sec_app_mths_since_last_major_derog',
         'settlement_percentage', 'settlement_term', 'debt_settlement_flag_date', 'settlement_status', 'settlement_amount',
         'settlement_date', 'disbursement_method', 'deferral_term', 'hardship_flag', 'debt_settlement_flag']

In [4]:
#creating a new dataframe without these columns
newdf = df.drop(columns=gendrops)

In [5]:
#creating our target feature based on defaulting
newdf['default'] = (newdf['loan_status'] == 'Default' ) | (newdf['loan_status'] == 'Charged Off')


I want to use my employment length column but it's currently filled with strings. I am going to convert it to integers.

In [6]:
#Median was found in a previous notebook.
newdf['emp_length'].fillna('6', inplace=True)

In [7]:
#function to change our emp length to usable integers as this information should be useful 
def experience_convert(s):
    if '<' in s:
        return 0
    elif '10+' in s:
        return 10
    else:
        for i in range(0,10):
            if str(i) in s:
                return i

In [8]:
#creating a list to make the column changes 
newemp = []
for i in newdf['emp_length']:
    newemp.append(experience_convert(i))

In [9]:
#making the actual changes to the column
newdf['emp_length'] = newemp

I have other columns that I think will be useful for my project but not necessarily useful for logistic regression. I am going to go ahead and toss them out too. 

In [10]:
#removing columns that are strings or that I otherwise think wont be useful for this regression
logdrops = ['term', 'grade', 'sub_grade', 'emp_title','home_ownership', 'verification_status', 'issue_d',
           'loan_status', 'pymnt_plan', 'purpose', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status',
           'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'application_type', 'verification_status_joint', 'sec_app_earliest_cr_line']

In [11]:
#drop more columns 
newdf = newdf.drop(columns=logdrops)

We can't have missing values for our logistic regression so let's see where we're at. 

In [12]:
missing = newdf.isna().sum()
missing = missing[missing > 0].sort_values(ascending=True)
100*missing/len(newdf)

inq_last_6mths                         0.000077
num_rev_accts                          0.000077
avg_cur_bal                            0.001386
dti                                    0.044570
revol_util                             0.066970
mths_since_recent_bc                   1.047279
bc_open_to_buy                         1.104011
percent_bc_gt_75                       1.128721
bc_util                                1.155432
mo_sin_old_il_acct                     2.854556
num_tl_120dpd_2m                       4.838882
mths_since_recent_inq                 10.933498
max_bal_bc                            30.774370
open_rv_24m                           30.774370
open_rv_12m                           30.774370
open_act_il                           30.774370
open_il_24m                           30.774370
open_il_12m                           30.774370
total_bal_il                          30.774370
inq_fi                                30.774370
inq_last_12m                          30

I still have a lot of missing values. I'm going to put the median in the ones that have around ~30% missing values and drop the rest.

In [13]:
#these are the columns where I want to fill in the missing values
fixlist = missing.index[:24].tolist()

In [14]:
#create a function to fill missing values with the median
def fixempty(col):
    med = newdf[col].median()
    newdf[col].fillna(med, inplace=True)

for i in fixlist:
    fixempty(i)

In [15]:
#these are the columns that I am just going to get rid of. 
empties = missing.index[24:].tolist()
newerdf = newdf.drop(columns=empties)

In [19]:
newerdf['default'] = newerdf['default']*1

I am still having issues getting my logistic regression to run. I believe the cause is likely that I have too much covariance. I am going to start dropping columns until I can get this thing to run. 

In [23]:
corr = newerdf.corr()

In [24]:
corr[corr > .7]

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,default
loan_amnt,1.000000,NaN,0.945876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.812690,0.812722,0.710123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_rate,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installment,0.945876,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.801043,0.801017,0.722438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_length,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dti,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delinq_2yrs,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fico_range_low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fico_range_high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Finally, we get to the actual logistic regression. 

In [21]:
#its screwed up but I am not sure how to make it work. 
'''logit_model=sm.Logit(newerdf['default'].values, newerdf.drop(columns=['default']).values)
result=logit_model.fit()
print(result.summary2())'''

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [33]:
Xlr, Xtestlr, ylr, ytestlr = train_test_split(newerdf.drop(columns=['default']), 
                                              newerdf['default'].values,random_state=10)

In [34]:
clf = LogisticRegression(solver='lbfgs')
# Fit the model on the trainng data.
clf.fit(Xlr, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))

C:\Users\hunte\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.9988699729963574


In [32]:
score = clf.score(Xtestlr, ytestlr)
print(score)

0.9987591256608502
